In [3]:
import tensorflow as tf
import scipy, numpy

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

In [4]:
# Configurações para o treinamento
train_data_dir = './data'
batch_size = 32 # 32 imagens por lote
num_epochs = 50 # 50 épocas de treinamento
image_size = (150, 150) # 150x150 pixels
num_classes = 2 # 2 classes: porco e gato

In [5]:
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    validation_split = 0.25, # 25% para validação
    shear_range = 0.2, # inclinação
    zoom_range = 0.2, # zoom
    horizontal_flip = True # espelhamento horizontal
)

In [6]:
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'training'
)

Found 16 images belonging to 2 classes.


In [7]:
validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = 'binary',
    subset = 'validation'
)

Found 4 images belonging to 2 classes.


In [8]:
model = Sequential() # instanciando o modelo
model.add(Conv2D(32, (3,3), input_shape=(image_size[0], image_size[1], 3), activation='relu')) # camada de convolução
model.add(MaxPooling2D(pool_size=(2,2))) # camada de pooling
model.add(Conv2D(64, (3,3), activation='relu')) # camada de convolução
model.add(MaxPooling2D(pool_size=(2,2))) # camada de pooling
model.add(Flatten()) # camada de achatamento ou vetorização
model.add(Dense(64, activation='relu')) # neurônios
model.add(Dropout(0.5)) # regularização 50% chance de desligar neurônios
model.add(Dense(1, activation='sigmoid')) # camada de saída (classificação binária)

In [9]:
# compilar o modelo
model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy', # mais próximo de 0 melhor, 0.002 por exemplo
    metrics = ['accuracy']
)

In [11]:
# treinamento
model.fit(
    train_generator,
    epochs = num_epochs,
    validation_data = validation_generator
)

print('Treinamento concluído!')

# salvar o modelo
model.save('models/cfModeloGersonRamos.h5')

Epoch 1/50
1/1 [==============================] - 0s 339ms/step - loss: 0.0100 - accuracy: 1.0000 - val_loss: 0.0914 - val_accuracy: 1.0000
Epoch 2/50
1/1 [==============================] - 0s 296ms/step - loss: 0.0182 - accuracy: 1.0000 - val_loss: 0.0334 - val_accuracy: 1.0000
Epoch 3/50
1/1 [==============================] - 0s 287ms/step - loss: 0.0227 - accuracy: 1.0000 - val_loss: 0.0457 - val_accuracy: 1.0000
Epoch 4/50
1/1 [==============================] - 0s 287ms/step - loss: 0.0247 - accuracy: 1.0000 - val_loss: 0.2934 - val_accuracy: 0.7500
Epoch 5/50
1/1 [==============================] - 0s 292ms/step - loss: 0.0175 - accuracy: 1.0000 - val_loss: 0.0359 - val_accuracy: 1.0000
Epoch 6/50
1/1 [==============================] - 0s 281ms/step - loss: 0.0030 - accuracy: 1.0000 - val_loss: 0.0173 - val_accuracy: 1.0000
Epoch 7/50
1/1 [==============================] - 0s 289ms/step - loss: 0.0108 - accuracy: 1.0000 - val_loss: 0.0450 - val_accuracy: 1.0000
Epoch 8/50
1/1 [====

C:\Users\Gerson\AppData\Roaming\Python\Python310\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
